In [1]:
from IPython.display import display, HTML, Image, clear_output
import time
import random
from time import sleep
from ipywidgets import Output
from bs4 import BeautifulSoup
import requests
import json
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

In [2]:
def send_to_google_form(data_dict, form_url):
    
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    
    return post_result.ok

In [3]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()
    
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            ui_poll(n_proc)

            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            time.sleep(interval)
    
    return event_info

def register_event(btn):
    
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    
    return

In [4]:
# lists of questions and answers
lev1 = [
    ("What object was in the top left corner?", "triangle"),
    ("What colour was the circle?", "purple"),
    ("What object was in blue?", "square"),
    ("What colour was the star?", "pink"),
    ("What object was above the star?", "circle")
]

lev2 = [
    ("What object was in the centre?", "pentagon"),
    ("What colour was the object in the bottom right corner?", "red"),
    ("What object was beside the arrow?", "rectangle"),
    ("What object was orange in colour?", "heart"),
    ("What colour was the object below the star?", "white")
]

lev3 = [
    ("What object was next to the heart?", "cherry"),
    ("What colour was the object next to the peach?", "green"),
    ("What object was in the bottom left corner?", "apple"),
    ("How many different fruits were there in the picture? (Type your answer in words)", "five"),
    ("What object was to the left of the object in pink?", "circle")
] 

def memory_test(): 
    """
    This test starts by asking for some information from the respondent for data analysis. 
    The test itself has 3 levels in total, in increasing difficulty.
    For each level, the respondent will have 15 seconds to observe a picture.
    After 15 seconds, they will then have to answer 5 questions about the picture.
    Once all 3 levels are completed, they are asked to provide consent for uploading their data to our repository.
    """

    # dictionary for saving responses
    results_dict = {
        "id": [],
        "gender": [],
        "age": [],
        "substance": [],
        "counter1": [],
        "counter2": [],
        "counter3": [],
        "final_score": [],
        "time_taken1": [],
        "time_taken2": [],
        "time_taken3": [],
        "total_time": [],
    }

    # start of test asking for respondent information
    display(HTML("<h3>Welcome to the Memory Test!</h3>"))
    time.sleep(1)

    display(HTML("<h4>Please enter an anonymised ID:</h4>"))
    time.sleep(1)
    display(HTML("<h5>To generate an anonymous 4-letter unique user identifier please enter:</h5>"))
    time.sleep(1)
    display(HTML("<h5>- Two letters based on the initials (first and last name) of a childhood friend</h5>"))
    time.sleep(1)
    display(HTML("<h5>- Two letters based on the initials (first and last name) of a favourite actor/actress</h5>"))
    time.sleep(1)
    display(HTML("<h5>E.g. If your friend was called Charlie Brown and film star was Tom Cruise</h5>"))
    time.sleep(1)
    display(HTML("<h5>Then your unique identifier would be CBTC</h5>"))
    time.sleep(1)
    display(HTML("<h5>Note: Please use the same ID for all Group 6 cognitive tests</h5>"))
    ans1 = input("> ")

    display(HTML("<h4>Please enter your gender:</h4>"))
    ans2 = input("> ")

    display(HTML("<h4>Please enter your age (in numbers):</h4>"))
    ans3 = input("> ")

    display(HTML("<h4>Have you consumed any substances (Eg. caffeine, alcohol, or drugs) in the last 12 hours that might affect your cognitive abilities?</h4>"))
    time.sleep(1)
    display(HTML("<h5>Please enter 'yes' or 'no'. If yes, please specify. Press enter if you would prefer not to disclose this information.</h5>"))
    ans4 = input("> ")

    display(HTML("<h4>Press enter when you are ready to continue</h4>"))
    input("")

    clear_output(wait=False)

    # explaining test to respondent
    display(HTML("<h4>There will be 3 levels in total</h4>"))
    time.sleep(1)

    display(HTML("<h4>For each level, you will have 15 seconds to commit a picture to memory</h4>"))
    time.sleep(1)
    
    display(HTML("<h4>You will then answer 5 questions about the picture</h4>"))
    time.sleep(1)

    display(HTML("<h4>Please read all the questions carefully and type all your answers in lowercase</h4>"))
    time.sleep(1)

    display(HTML("<h4>Press enter when you are ready to continue</h4>"))
    input("")

    # start of level 1
    level1 = Image("memory1.png", width = 400)
    out1 = Output() # for countdown timer

    display(level1)
    display(out1)

    counter1 = 0 # to keep track of scores in each separate level

    # displaying countdown timer
    for i in range (15, -1, -1):
        out1.clear_output(wait=True)
        with out1: display(HTML(f"<h4>Time remaining: {i} seconds</h4>"))

        time.sleep(1)

    clear_output(wait=False)

    # randomise questions
    lev1_ques = lev1.copy()
    random.seed(1)
    random.shuffle(lev1_ques)

    # keep track of time taken to answer questions
    start_time1 = time.time()

    for question, correct_ans in lev1_ques:
        display(HTML(f"<h4>{question}</h4>"))
        answer = input("> ")
        if answer == correct_ans:
            display(HTML("<h4>Correct!</h4>"))
            time.sleep(2)
            clear_output(wait=False)
            counter1 += 1
        else:
            display(HTML(f"<h4>Sorry! The answer is {correct_ans!r}, not {answer!r}</h4>"))
            time.sleep(2)
            clear_output(wait=False)

    end_time1 = time.time()
    time_taken1 = end_time1 - start_time1

    display(HTML("<h4>Congratulations! You have completed Level 1</h4>"))
    time.sleep(1)

    # start of level 2
    level2 = Image("memory2.png", width = 400)
    
    display(HTML("<h4>Welcome to Level 2</h4>"))
    time.sleep(1)

    display(HTML("<h4>Press enter when you are ready to continue</h4>"))
    input("")

    display(level2)
    display(out1)

    counter2 = 0

    for i in range (15, -1, -1):
        out1.clear_output(wait=True)
        with out1: display(HTML(f"<h4>Time remaining: {i} seconds</h4>"))

        time.sleep(1)

    clear_output(wait=False)

    lev2_ques = lev2.copy()
    random.seed(1)
    random.shuffle(lev2_ques)

    start_time2 = time.time()

    for question, correct_ans in lev2_ques:
        display(HTML(f"<h4>{question}</h4>"))
        answer = input("> ")
        if answer == correct_ans:
            display(HTML("<h4>Correct!</h4>"))
            time.sleep(2)
            clear_output(wait=False)
            counter2 += 1
        else:
            display(HTML(f"<h4>Sorry! The answer is {correct_ans!r}, not {answer!r}</h4>"))
            time.sleep(2)
            clear_output(wait=False)

    end_time2 = time.time()
    time_taken2 = end_time2 - start_time2

    display(HTML("<h4>Congratulations! You have completed Level 2</h4>"))
    time.sleep(1)

    # start of level 3
    level3 = Image("memorypic3.png", width = 400)
    
    display(HTML("<h4>Welcome to Level 3</h4>"))
    time.sleep(1)

    display(HTML("<h4>Press enter when you are ready to continue</h4>"))
    input("")

    display(level3)
    display(out1)

    counter3 = 0

    for i in range (15, -1, -1):
        out1.clear_output(wait=True)
        with out1: display(HTML(f"<h4>Time remaining: {i} seconds</h4>"))

        time.sleep(1)

    clear_output(wait=False)

    lev3_ques = lev3.copy()
    random.seed(1)
    random.shuffle(lev3_ques)

    start_time3 = time.time()

    for question, correct_ans in lev3_ques:
        display(HTML(f"<h4>{question}</h4>"))
        answer = input("> ")
        if answer == correct_ans:
            display(HTML("<h4>Correct!</h4>"))
            time.sleep(2)
            clear_output(wait=False)
            counter3 += 1
        else:
            display(HTML(f"<h4>Sorry! The answer is {correct_ans!r}, not {answer!r}</h4>"))
            time.sleep(2)
            clear_output(wait=False)

    end_time3 = time.time()
    time_taken3 = end_time3 - start_time3

    # calculate total time taken and final score across all three levels
    total_time = time_taken1 + time_taken2 + time_taken3
    final_score = counter1 + counter2 + counter3

    display(HTML("<h4>Thank you for taking the test!</h4>"))
    time.sleep(1)
    display(HTML(f"<h4>Your final score is: {final_score}/15</h4>"))
    time.sleep(1)
    display(HTML(f"<h4>The total time taken was: {total_time:.2f} seconds</h4>"))
    time.sleep(1)

    # ask for consent for uploading data
    print("")
    time.sleep(1)
    display(HTML("<h4>Please read:</h3>"))
    time.sleep(1)
    display(HTML("<h4>We wish to record your response data</h4>"))
    time.sleep(1)
    display(HTML("<h4>to an anonymised public data repository.</h4>"))
    time.sleep(1)
    display(HTML("<h4>Your data will be used for educational teaching purposes</h4>"))
    time.sleep(1)
    display(HTML("<h4>practising data analysis and visualisation.</h4>"))
    time.sleep(1)
    display(HTML("<h4>Please select 'yes' in the box below if you consent to the upload.</h4>"))
    time.sleep(1)
    display(HTML("<h4>You have 15 seconds to answer.</h4>"))
    time.sleep(1)

    # define variables for buttons
    btn1 = widgets.Button(description="Yes")
    btn2 = widgets.Button(description="No")
    
    btn1.on_click(register_event) 
    btn2.on_click(register_event)

    panel = widgets.HBox([btn1, btn2])
    display(panel)

    consent_result = wait_for_event(timeout=15)
    clear_output()

    # upload results onto google forms
    if consent_result['description']=="Yes":
        results_dict["id"].append(ans1)
        results_dict["gender"].append(ans2)
        results_dict["age"].append(ans3)
        results_dict["substance"].append(ans4)
        results_dict["counter1"].append(counter1)
        results_dict["counter2"].append(counter2)
        results_dict["counter3"].append(counter3)
        results_dict["final_score"].append(final_score)
        results_dict["time_taken1"].append(time_taken1)
        results_dict["time_taken2"].append(time_taken2)
        results_dict["time_taken3"].append(time_taken3)
        results_dict["total_time"].append(total_time)

        form_url = "https://docs.google.com/forms/d/e/1FAIpQLScRVOFAri5zqvY2SQyOptEvTkXmeukUxcjXwE_p-uysQV-kKw/viewform?usp=sf_link"
        send_to_google_form(results_dict, form_url)

        display(HTML("<h4>Thank you for your participation! Your data has been uploaded.</h4>"))
        display(HTML("<h4>Please contact a.fedorec@ucl.ac.uk</h4>"))
        display(HTML("<h4>If you have any questions or concerns regarding the stored results.</h4>"))
    # results are not uploaded if respondent did not select yes
    else:
        display(HTML("<h4>That's okay! Thank you for taking the test.</h4>"))

    return 

In [5]:
memory_test()